In [28]:
import os
import shutil
from pathlib import Path

def organize_by_sigma(source_dir):
    """
    Organizes images in the source directory into subdirectories based on sigma levels.
    
    Args:
        source_dir (str): Path to the directory containing the images
        
    The function will:
    1. Create subdirectories for each sigma level
    2. Move each image to its corresponding sigma directory
    3. Print progress and summary information
    """
    # Convert the source directory to a Path object for easier handling
    source_path = Path(source_dir)
    
    # Dictionary to keep track of how many files are moved to each sigma level
    files_moved = {}
    
    # Iterate through all PNG files in the source directory
    for image_file in source_path.glob('*.png'):
        try:
            # Extract the sigma level from the filename
            # Filename format: XX_sigma_YY.png where YY is the sigma level
            sigma_level = image_file.name.split('sigma_')[1].split('.')[0]
            
            # Create the sigma level directory if it doesn't exist
            sigma_dir = source_path / f"sigma_{sigma_level}"
            sigma_dir.mkdir(exist_ok=True)
            
            # Move the file to its sigma directory
            shutil.move(str(image_file), str(sigma_dir / image_file.name))
            
            # Update our counter
            files_moved[sigma_level] = files_moved.get(sigma_level, 0) + 1
            
        except Exception as e:
            print(f"Error processing {image_file.name}: {str(e)}")
    
    # Print summary
    print("\nOrganization complete! Summary:")
    for sigma, count in sorted(files_moved.items(), key=lambda x: int(x[0])):
        print(f"Sigma {sigma}: {count} images moved")
    
    return files_moved

# Path to your directory containing the images
source_directory = "E:\LinearStyleTransfer-master\LinearStyleTransfer\data\styled_outputs\style_21"  # Replace with your actual path

if __name__ == "__main__":
    # Check if the source directory exists
    if not os.path.exists(source_directory):
        print(f"Error: Directory '{source_directory}' does not exist!")
    else:
        print(f"Starting to organize images in: {source_directory}")
        organize_by_sigma(source_directory)

Starting to organize images in: E:\LinearStyleTransfer-master\LinearStyleTransfer\data\styled_outputs\style_21

Organization complete! Summary:
Sigma 5: 16 images moved
Sigma 10: 16 images moved
Sigma 15: 16 images moved
Sigma 20: 16 images moved
Sigma 25: 16 images moved
Sigma 30: 16 images moved
Sigma 40: 16 images moved
Sigma 45: 16 images moved
Sigma 50: 16 images moved


<>:51: SyntaxWarning: invalid escape sequence '\L'
<>:51: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Ishaan\AppData\Local\Temp\ipykernel_6796\2223061107.py:51: SyntaxWarning: invalid escape sequence '\L'
  source_directory = "E:\LinearStyleTransfer-master\LinearStyleTransfer\data\styled_outputs\style_21"  # Replace with your actual path


In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
import os

# Step 1: Load TensorFlow Model and Extract Weights
# Define paths
model_path = '/home/sunayana/Documents/LinearStyleTransfer/models/vgg_r11/tensorflow2-decoder-v1'
weights_dir = "./weights/dec_r11"
os.makedirs(weights_dir, exist_ok=True)

print("Loading TensorFlow model...")
loaded_model = tf.saved_model.load(model_path)

print("Extracting and saving weights...")
# Iterate over the model variables and save weights as NumPy arrays
for var in loaded_model.variables:
    weight_name = var.name.replace("/", "_").replace(":", "_")  # Clean up the variable name for saving
    np.save(os.path.join(weights_dir, f"{weight_name}.npy"), var.numpy())
    print(f"Saved {weight_name} with shape {var.shape}")

# Step 2: Define the Equivalent PyTorch Decoder Model
print("\nDefining PyTorch model...")

class VGGDecoderR11(nn.Module):
    def __init__(self):
        super(VGGDecoderR11, self).__init__()
        # Example model architecture for the decoder
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.conv1_1 = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.relu1_1 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1_2(x)
        x = self.relu1_2(x)
        x = self.conv1_1(x)
        x = self.relu1_1(x)
        return x

# Initialize PyTorch model
torch_model = VGGDecoderR11()

# Step 3: Load the Weights into the PyTorch Model
print("\nLoading weights into PyTorch model...")

def load_weights(torch_model, weights_dir):
    # Weight files mapping according to extracted TensorFlow variables
    weight_files = {
        "conv1_2.weight": "rev_block1_conv2_kernel_0.npy",
        "conv1_2.bias": "rev_block1_conv2_bias_0.npy",
        "conv1_1.weight": "rev_block1_conv1_kernel_0.npy",
        "conv1_1.bias": "rev_block1_conv1_bias_0.npy"
    }

    for param_name, npy_file in weight_files.items():
        # Load the numpy file
        param_weight = torch.from_numpy(np.load(os.path.join(weights_dir, npy_file)))
        
        # Transpose if needed to match PyTorch dimension format
        if 'weight' in param_name and len(param_weight.shape) == 4:
            # TensorFlow uses (H, W, in_channels, out_channels) whereas PyTorch uses (out_channels, in_channels, H, W)
            param_weight = param_weight.permute(3, 2, 0, 1)
        
        # Get the parameter (either weight or bias) from the model
        param = getattr(torch_model, param_name.split(".")[0])
        
        # Assign the weights to the PyTorch layer
        if 'weight' in param_name:
            param.weight.data.copy_(param_weight)
        elif 'bias' in param_name:
            param.bias.data.copy_(param_weight)
        
        print(f"Loaded weights for {param_name}")

# Load weights into the PyTorch model
load_weights(torch_model, weights_dir)

print("\nConversion complete!")

# Step 4: Save the PyTorch Model Weights
output_path = "models/dec_r11.pth"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
torch.save(torch_model.state_dict(), output_path)
print(f"\nModel weights saved to {output_path}")

# Optional: Verify the output
print("\nVerifying the output...")

# Create a random input tensor to test the model
input_tensor = torch.randn(1, 64, 224, 224)  # Batch size 1, 64 channels, 224x224 image
output = torch_model(input_tensor)

print("Output shape:", output.shape)


2024-11-06 04:48:25.974844: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-06 04:48:25.978124: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-06 04:48:25.988231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730848706.005246  278137 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730848706.010372  278137 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 04:48:26.028304: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Loading TensorFlow model...
Extracting and saving weights...
Saved rev_block1_conv2_kernel_0 with shape (3, 3, 64, 64)
Saved rev_block1_conv2_bias_0 with shape (64,)
Saved rev_block1_conv1_kernel_0 with shape (3, 3, 64, 3)
Saved rev_block1_conv1_bias_0 with shape (3,)

Defining PyTorch model...

Loading weights into PyTorch model...
Loaded weights for conv1_2.weight
Loaded weights for conv1_2.bias
Loaded weights for conv1_1.weight
Loaded weights for conv1_1.bias

Conversion complete!

Model weights saved to models/dec_r11.pth

Verifying the output...
Output shape: torch.Size([1, 3, 224, 224])


W0000 00:00:1730848708.996150  278137 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
import torch
import numpy as np
import os

# Directory containing the extracted weights in .npy files
weights_dir = "./weights/dec_r11"  # Replace with the correct path to your weights directory

# List of numpy weight files and corresponding PyTorch model parameters
weight_files = {
    "conv1_2.weight": "rev_block1_conv2_kernel_0.npy",
    "conv1_2.bias": "rev_block1_conv2_bias_0.npy",
    "conv1_1.weight": "rev_block1_conv1_kernel_0.npy",
    "conv1_1.bias": "rev_block1_conv1_bias_0.npy"
}

# Step 1: Create an empty PyTorch state dictionary
state_dict = {}

# Step 2: Iterate over the weight files and add them to the state dictionary
print("Loading numpy files and converting to PyTorch tensors...")
for param_name, npy_file in weight_files.items():
    # Construct the full path to the .npy file
    file_path = os.path.join(weights_dir, npy_file)
    
    # Load the numpy array from file
    numpy_array = np.load(file_path)
    
    # Convert numpy array to PyTorch tensor
    tensor = torch.from_numpy(numpy_array)

    # Transpose if necessary to match PyTorch's dimension format
    if "weight" in param_name and len(tensor.shape) == 4:  # Convolutional weights are 4D
        tensor = tensor.permute(3, 2, 0, 1)  # TensorFlow uses (H, W, In_C, Out_C), PyTorch uses (Out_C, In_C, H, W)

    # Add tensor to the state dictionary
    state_dict[param_name] = tensor
    print(f"Loaded {param_name} with shape {tensor.shape}")

# Step 3: Save the state dictionary to a .pth file
output_path = "./models/dec_r11.pth"  # Replace with your desired output path
os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Create the directory if it doesn't exist

torch.save(state_dict, output_path)
print(f"\nModel weights saved to {output_path}")


Loading numpy files and converting to PyTorch tensors...
Loaded conv1_2.weight with shape torch.Size([64, 64, 3, 3])
Loaded conv1_2.bias with shape torch.Size([64])
Loaded conv1_1.weight with shape torch.Size([3, 64, 3, 3])
Loaded conv1_1.bias with shape torch.Size([3])

Model weights saved to ./models/dec_r11.pth


In [3]:
import tensorflow as tf
import numpy as np
import os

# Define paths
model_path = '/home/sunayana/Documents/LinearStyleTransfer/models/vgg_r21/tensorflow-encoder-v1'  # Replace with your TensorFlow model path
weights_dir = "./weights/vgg_r21"  # Directory to store the weights as .npy files
os.makedirs(weights_dir, exist_ok=True)

# Step 1: Load TensorFlow Model and Extract Weights
print("Loading TensorFlow model...")
loaded_model = tf.saved_model.load(model_path)

print("Extracting and saving weights...")
# Iterate over the model variables and save weights as NumPy arrays
for var in loaded_model.variables:
    weight_name = var.name.replace("/", "_").replace(":", "_")  # Clean up the variable name for saving
    np.save(os.path.join(weights_dir, f"{weight_name}.npy"), var.numpy())
    print(f"Saved {weight_name} with shape {var.shape}")



Loading TensorFlow model...
Extracting and saving weights...
Saved block1_conv1_kernel_0 with shape (3, 3, 3, 64)
Saved block1_conv1_bias_0 with shape (64,)
Saved block1_conv2_kernel_0 with shape (3, 3, 64, 64)
Saved block1_conv2_bias_0 with shape (64,)
Saved block2_conv1_kernel_0 with shape (3, 3, 64, 128)
Saved block2_conv1_bias_0 with shape (128,)
Saved block2_conv2_kernel_0 with shape (3, 3, 128, 128)
Saved block2_conv2_bias_0 with shape (128,)


In [4]:
import torch
import torch.nn as nn
import numpy as np
import os

# Step 1: Define the Equivalent PyTorch Encoder Model for r21
print("\nDefining PyTorch model...")

class VGGEncoderR21(nn.Module):
    def __init__(self):
        super(VGGEncoderR21, self).__init__()
        # Example model architecture matching the TensorFlow model for r21 encoder
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        # You can add more layers if required, depending on the TensorFlow model

    def forward(self, x):
        x = self.conv1_1(x)
        x = self.relu1_1(x)
        x = self.conv1_2(x)
        x = self.relu1_2(x)
        x = self.maxpool1(x)
        x = self.conv2_1(x)
        x = self.relu2_1(x)
        return x

# Initialize PyTorch model
torch_model = VGGEncoderR21()

# Step 2: Load the Weights into the PyTorch Model
print("\nLoading weights into PyTorch model...")

# Directory containing the extracted weights in .npy files
weights_dir = "./weights/vgg_r21"  # Replace with your path to the weights directory

# List of numpy weight files and corresponding PyTorch model parameters
weight_files = {
    "conv1_1.weight": "block1_conv1_kernel_0.npy",
    "conv1_1.bias": "block1_conv1_bias_0.npy",
    "conv1_2.weight": "block1_conv2_kernel_0.npy",
    "conv1_2.bias": "block1_conv2_bias_0.npy",
    "conv2_1.weight": "block2_conv1_kernel_0.npy",
    "conv2_1.bias": "block2_conv1_bias_0.npy"
}

# Step 3: Create an empty PyTorch state dictionary
state_dict = {}

# Step 4: Iterate over the weight files and add them to the state dictionary
for param_name, npy_file in weight_files.items():
    # Construct the full path to the .npy file
    file_path = os.path.join(weights_dir, npy_file)
    
    # Load the numpy array from file
    numpy_array = np.load(file_path)
    
    # Convert numpy array to PyTorch tensor
    tensor = torch.from_numpy(numpy_array)

    # Transpose if necessary to match PyTorch's dimension format
    if "weight" in param_name and len(tensor.shape) == 4:  # Convolutional weights are 4D
        tensor = tensor.permute(3, 2, 0, 1)  # TensorFlow uses (H, W, In_C, Out_C), PyTorch uses (Out_C, In_C, H, W)

    # Add tensor to the state dictionary
    state_dict[param_name] = tensor
    print(f"Loaded {param_name} with shape {tensor.shape}")

# Step 5: Load the state dictionary into the PyTorch model
torch_model.load_state_dict(state_dict)

# Step 6: Save the state dictionary to a .pth file
output_path = "./models/vgg_r21.pth"  # Replace with your desired output path
os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Create the directory if it doesn't exist

torch.save(state_dict, output_path)
print(f"\nModel weights saved to {output_path}")

# Optional: Verify the output
print("\nVerifying the output...")
# Create a random input tensor to test the model
input_tensor = torch.randn(1, 3, 224, 224)  # Batch size 1, 3 channels, 224x224 image
output = torch_model(input_tensor)
print("Output shape:", output.shape)



Defining PyTorch model...

Loading weights into PyTorch model...
Loaded conv1_1.weight with shape torch.Size([64, 3, 3, 3])
Loaded conv1_1.bias with shape torch.Size([64])
Loaded conv1_2.weight with shape torch.Size([64, 64, 3, 3])
Loaded conv1_2.bias with shape torch.Size([64])
Loaded conv2_1.weight with shape torch.Size([128, 64, 3, 3])
Loaded conv2_1.bias with shape torch.Size([128])

Model weights saved to ./models/vgg_r21.pth

Verifying the output...
Output shape: torch.Size([1, 128, 112, 112])


In [6]:
import tensorflow as tf
import numpy as np
import os

# Define paths
model_path = '/home/sunayana/Documents/LinearStyleTransfer/models/vgg_r21/tensorflow-decoder-v1'  # Replace with your TensorFlow model path
weights_dir = "./weights/dec_r21"  # Directory to store the weights as .npy files
os.makedirs(weights_dir, exist_ok=True)

# Step 1: Load TensorFlow Model and Extract Weights
print("Loading TensorFlow model...")
loaded_model = tf.saved_model.load(model_path)

print("Extracting and saving weights...")
# Iterate over the model variables and save weights as NumPy arrays
for var in loaded_model.variables:
    weight_name = var.name.replace("/", "_").replace(":", "_")  # Clean up the variable name for saving
    np.save(os.path.join(weights_dir, f"{weight_name}.npy"), var.numpy())
    print(f"Saved {weight_name} with shape {var.shape}")


Loading TensorFlow model...
Extracting and saving weights...
Saved rev_block2_conv2_kernel_0 with shape (3, 3, 128, 128)
Saved rev_block2_conv2_bias_0 with shape (128,)
Saved rev_block2_conv1_kernel_0 with shape (3, 3, 128, 64)
Saved rev_block2_conv1_bias_0 with shape (64,)
Saved rev_block1_conv2_kernel_0 with shape (3, 3, 64, 64)
Saved rev_block1_conv2_bias_0 with shape (64,)
Saved rev_block1_conv1_kernel_0 with shape (3, 3, 64, 3)
Saved rev_block1_conv1_bias_0 with shape (3,)


In [7]:
import torch
import torch.nn as nn
import numpy as np
import os

# Step 1: Define the Equivalent PyTorch Decoder Model for r21
print("\nDefining PyTorch model...")

class VGGDecoderR21(nn.Module):
    def __init__(self):
        super(VGGDecoderR21, self).__init__()
        # Example model architecture matching the TensorFlow model for r21 decoder
        self.conv2_1 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        
        self.unpool1 = nn.UpsamplingNearest2d(scale_factor=2)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        
        self.conv1_1 = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1)
        # You can add more layers if required, depending on the TensorFlow model

    def forward(self, x):
        x = self.conv2_1(x)
        x = self.relu2_1(x)
        x = self.unpool1(x)
        x = self.conv1_2(x)
        x = self.relu1_2(x)
        x = self.conv1_1(x)
        return x

# Initialize PyTorch model
torch_model = VGGDecoderR21()

# Step 2: Load the Weights into the PyTorch Model
print("\nLoading weights into PyTorch model...")

# Directory containing the extracted weights in .npy files
weights_dir = "./weights/dec_r21"  # Replace with your path to the weights directory

# List of numpy weight files and corresponding PyTorch model parameters
weight_files = {
    "conv2_1.weight": "rev_block2_conv1_kernel_0.npy",
    "conv2_1.bias": "rev_block2_conv1_bias_0.npy",
    "conv1_2.weight": "rev_block1_conv2_kernel_0.npy",
    "conv1_2.bias": "rev_block1_conv2_bias_0.npy",
    "conv1_1.weight": "rev_block1_conv1_kernel_0.npy",
    "conv1_1.bias": "rev_block1_conv1_bias_0.npy"
}

# Step 3: Create an empty PyTorch state dictionary
state_dict = {}

# Step 4: Iterate over the weight files and add them to the state dictionary
for param_name, npy_file in weight_files.items():
    # Construct the full path to the .npy file
    file_path = os.path.join(weights_dir, npy_file)
    
    # Load the numpy array from file
    numpy_array = np.load(file_path)
    
    # Convert numpy array to PyTorch tensor
    tensor = torch.from_numpy(numpy_array)

    # Transpose if necessary to match PyTorch's dimension format
    if "weight" in param_name and len(tensor.shape) == 4:  # Convolutional weights are 4D
        tensor = tensor.permute(3, 2, 0, 1)  # TensorFlow uses (H, W, In_C, Out_C), PyTorch uses (Out_C, In_C, H, W)

    # Add tensor to the state dictionary
    state_dict[param_name] = tensor
    print(f"Loaded {param_name} with shape {tensor.shape}")

# Step 5: Load the state dictionary into the PyTorch model
torch_model.load_state_dict(state_dict)

# Step 6: Save the state dictionary to a .pth file
output_path = "./models/dec_r21.pth"  # Replace with your desired output path
os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Create the directory if it doesn't exist

torch.save(state_dict, output_path)
print(f"\nModel weights saved to {output_path}")

# Optional: Verify the output
print("\nVerifying the output...")
# Create a random input tensor to test the model
input_tensor = torch.randn(1, 128, 56, 56)  # Batch size 1, 128 channels, 56x56 feature map
output = torch_model(input_tensor)
print("Output shape:", output.shape)



Defining PyTorch model...

Loading weights into PyTorch model...
Loaded conv2_1.weight with shape torch.Size([64, 128, 3, 3])
Loaded conv2_1.bias with shape torch.Size([64])
Loaded conv1_2.weight with shape torch.Size([64, 64, 3, 3])
Loaded conv1_2.bias with shape torch.Size([64])
Loaded conv1_1.weight with shape torch.Size([3, 64, 3, 3])
Loaded conv1_1.bias with shape torch.Size([3])

Model weights saved to ./models/dec_r21.pth

Verifying the output...
Output shape: torch.Size([1, 3, 112, 112])


In [9]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
import os

# Step 1: Load TensorFlow Model and Extract Weights
print("Loading TensorFlow model...")
model_path = '/home/sunayana/Documents/LinearStyleTransfer/models/vgg_r51/tensorflow-encoder-v1'
loaded_model = tf.saved_model.load(model_path)

# Directory to save numpy weights
weights_dir = "./weights/enc_r51"
os.makedirs(weights_dir, exist_ok=True)

print("Extracting and saving weights...")
# Extract weights from the TensorFlow model and save as .npy files
for var in loaded_model.variables:
    weight_name = var.name.replace("/", "_").replace(":", "_")  # Clean up variable names
    np.save(os.path.join(weights_dir, f"{weight_name}.npy"), var.numpy())
    print(f"Saved {weight_name} with shape {var.shape}")

# Step 2: Define the Equivalent PyTorch Model for Encoder r51
print("\nDefining PyTorch model...")

class VGGEncoderR51(nn.Module):
    def __init__(self):
        super(VGGEncoderR51, self).__init__()
        # Defining encoder layers equivalent to TensorFlow model
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.maxPool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.maxPool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.maxPool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.maxPool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1_1(x)
        x = self.relu1_1(x)
        x = self.conv1_2(x)
        x = self.relu1_2(x)
        x = self.maxPool1(x)

        x = self.conv2_1(x)
        x = self.relu2_1(x)
        x = self.conv2_2(x)
        x = self.relu2_2(x)
        x = self.maxPool2(x)

        x = self.conv3_1(x)
        x = self.relu3_1(x)
        x = self.conv3_2(x)
        x = self.relu3_2(x)
        x = self.conv3_3(x)
        x = self.relu3_3(x)
        x = self.maxPool3(x)

        x = self.conv4_1(x)
        x = self.relu4_1(x)
        x = self.conv4_2(x)
        x = self.relu4_2(x)
        x = self.conv4_3(x)
        x = self.relu4_3(x)
        x = self.maxPool4(x)

        x = self.conv5_1(x)
        x = self.relu5_1(x)

        return x

# Initialize the PyTorch model
torch_model = VGGEncoderR51()

# Step 3: Load Weights from NumPy Files into PyTorch Model
print("\nLoading weights into PyTorch model...")

def load_weights(torch_model, weights_dir):
    # Mapping of weight files to model parameter names
    weight_files = {
        "conv1_1.weight": "block1_conv1_kernel_0.npy",
        "conv1_1.bias": "block1_conv1_bias_0.npy",
        "conv1_2.weight": "block1_conv2_kernel_0.npy",
        "conv1_2.bias": "block1_conv2_bias_0.npy",
        "conv2_1.weight": "block2_conv1_kernel_0.npy",
        "conv2_1.bias": "block2_conv1_bias_0.npy",
        "conv2_2.weight": "block2_conv2_kernel_0.npy",
        "conv2_2.bias": "block2_conv2_bias_0.npy",
        "conv3_1.weight": "block3_conv1_kernel_0.npy",
        "conv3_1.bias": "block3_conv1_bias_0.npy",
        "conv3_2.weight": "block3_conv2_kernel_0.npy",
        "conv3_2.bias": "block3_conv2_bias_0.npy",
        "conv3_3.weight": "block3_conv3_kernel_0.npy",
        "conv3_3.bias": "block3_conv3_bias_0.npy",
        "conv4_1.weight": "block4_conv1_kernel_0.npy",
        "conv4_1.bias": "block4_conv1_bias_0.npy",
        "conv4_2.weight": "block4_conv2_kernel_0.npy",
        "conv4_2.bias": "block4_conv2_bias_0.npy",
        "conv4_3.weight": "block4_conv3_kernel_0.npy",
        "conv4_3.bias": "block4_conv3_bias_0.npy",
        "conv5_1.weight": "block5_conv1_kernel_0.npy",
        "conv5_1.bias": "block5_conv1_bias_0.npy"
    }

    # Load weights into PyTorch model
    for param_name, npy_file in weight_files.items():
        # Get parameter tensor from model
        param = getattr(torch_model, param_name.split(".")[0])
        param_weight = torch.from_numpy(np.load(os.path.join(weights_dir, npy_file)))

        # Transpose weights if it's a convolutional weight
        if 'weight' in param_name:
            param_weight = param_weight.permute(3, 2, 0, 1)  # (H, W, in_channels, out_channels) -> (out_channels, in_channels, H, W)

        # Assign the weights to the PyTorch layer
        if 'weight' in param_name:
            param.weight.data.copy_(param_weight)
        elif 'bias' in param_name:
            param.bias.data.copy_(param_weight)
        
        print(f"Loaded weights for {param_name}")

# Load weights into the PyTorch model
load_weights(torch_model, weights_dir)

print("\nConversion complete!")

# Step 4: Save the PyTorch Model State Dictionary
print("\nSaving PyTorch model state dictionary...")

output_path = "./models/enc_r51_updated.pth"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
torch.save(torch_model.state_dict(), output_path)
print(f"Model weights saved to {output_path}")

# Optional: Verify the output
print("\nVerifying the output...")
input_tensor = torch.randn(1, 3, 224, 224)  # Example input
output = torch_model(input_tensor)
print("Output shape:", output.shape)


Loading TensorFlow model...
Extracting and saving weights...
Saved block1_conv1_kernel_0 with shape (3, 3, 3, 64)
Saved block1_conv1_bias_0 with shape (64,)
Saved block1_conv2_kernel_0 with shape (3, 3, 64, 64)
Saved block1_conv2_bias_0 with shape (64,)
Saved block2_conv1_kernel_0 with shape (3, 3, 64, 128)
Saved block2_conv1_bias_0 with shape (128,)
Saved block2_conv2_kernel_0 with shape (3, 3, 128, 128)
Saved block2_conv2_bias_0 with shape (128,)
Saved block3_conv1_kernel_0 with shape (3, 3, 128, 256)
Saved block3_conv1_bias_0 with shape (256,)
Saved block3_conv2_kernel_0 with shape (3, 3, 256, 256)
Saved block3_conv2_bias_0 with shape (256,)
Saved block3_conv3_kernel_0 with shape (3, 3, 256, 256)
Saved block3_conv3_bias_0 with shape (256,)
Saved block3_conv4_kernel_0 with shape (3, 3, 256, 256)
Saved block3_conv4_bias_0 with shape (256,)
Saved block4_conv1_kernel_0 with shape (3, 3, 256, 512)
Saved block4_conv1_bias_0 with shape (512,)
Saved block4_conv2_kernel_0 with shape (3, 3, 5

In [13]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
import os

# Step 1: Load TensorFlow Model and Extract Weights
print("Loading TensorFlow model...")
model_path = '/home/sunayana/Documents/LinearStyleTransfer/models/vgg_r51/tensorflow-decoder-v1'
loaded_model = tf.saved_model.load(model_path)

# Directory to save numpy weights
weights_dir = "./weights/dec_r51"
os.makedirs(weights_dir, exist_ok=True)

print("Extracting and saving weights...")
# Extract weights from the TensorFlow model and save as .npy files
for var in loaded_model.variables:
    weight_name = var.name.replace("/", "_").replace(":", "_")  # Clean up variable names
    np.save(os.path.join(weights_dir, f"{weight_name}.npy"), var.numpy())
    print(f"Saved {weight_name} with shape {var.shape}")

# Step 2: Define the Equivalent PyTorch Model for Decoder r51
print("\nDefining PyTorch model...")

class VGGDecoderR51(nn.Module):
    def __init__(self):
        super(VGGDecoderR51, self).__init__()
        # Defining decoder layers equivalent to TensorFlow model
        self.reflecPad15 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv15 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=0)
        self.relu15 = nn.ReLU(inplace=True)

        self.unpool = nn.UpsamplingNearest2d(scale_factor=2)

        self.reflecPad16 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv16 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=0)
        self.relu16 = nn.ReLU(inplace=True)

        self.reflecPad17 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv17 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=0)
        self.relu17 = nn.ReLU(inplace=True)

        self.reflecPad18 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv18 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=0)
        self.relu18 = nn.ReLU(inplace=True)

        self.reflecPad19 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv19 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=0)
        self.relu19 = nn.ReLU(inplace=True)

        self.unpool2 = nn.UpsamplingNearest2d(scale_factor=2)

        self.reflecPad20 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv20 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=0)
        self.relu20 = nn.ReLU(inplace=True)

        self.reflecPad21 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv21 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=0)
        self.relu21 = nn.ReLU(inplace=True)

        self.reflecPad22 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv22 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=0)
        self.relu22 = nn.ReLU(inplace=True)

        self.reflecPad23 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv23 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=0)
        self.relu23 = nn.ReLU(inplace=True)

        self.unpool3 = nn.UpsamplingNearest2d(scale_factor=2)

        self.reflecPad24 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv24 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=0)
        self.relu24 = nn.ReLU(inplace=True)

        self.reflecPad25 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv25 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=0)
        self.relu25 = nn.ReLU(inplace=True)

        self.unpool4 = nn.UpsamplingNearest2d(scale_factor=2)

        self.reflecPad26 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv26 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0)
        self.relu26 = nn.ReLU(inplace=True)

        self.reflecPad27 = nn.ReflectionPad2d((1, 1, 1, 1))
        self.conv27 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=0)

    def forward(self, x):
        x = self.reflecPad15(x)
        x = self.conv15(x)
        x = self.relu15(x)
        x = self.unpool(x)

        x = self.reflecPad16(x)
        x = self.conv16(x)
        x = self.relu16(x)

        x = self.reflecPad17(x)
        x = self.conv17(x)
        x = self.relu17(x)

        x = self.reflecPad18(x)
        x = self.conv18(x)
        x = self.relu18(x)

        x = self.reflecPad19(x)
        x = self.conv19(x)
        x = self.relu19(x)
        x = self.unpool2(x)

        x = self.reflecPad20(x)
        x = self.conv20(x)
        x = self.relu20(x)

        x = self.reflecPad21(x)
        x = self.conv21(x)
        x = self.relu21(x)

        x = self.reflecPad22(x)
        x = self.conv22(x)
        x = self.relu22(x)

        x = self.reflecPad23(x)
        x = self.conv23(x)
        x = self.relu23(x)
        x = self.unpool3(x)

        x = self.reflecPad24(x)
        x = self.conv24(x)
        x = self.relu24(x)

        x = self.reflecPad25(x)
        x = self.conv25(x)
        x = self.relu25(x)
        x = self.unpool4(x)

        x = self.reflecPad26(x)
        x = self.conv26(x)
        x = self.relu26(x)

        x = self.reflecPad27(x)
        x = self.conv27(x)

        return x

# Initialize the PyTorch model
torch_model = VGGDecoderR51()

# Step 3: Load Weights from NumPy Files into PyTorch Model
print("\nLoading weights into PyTorch model...")

def load_weights(torch_model, weights_dir):
    # Mapping of weight files to model parameter names
    weight_files = {
        "conv15.weight": "rev_block5_conv1_kernel_0.npy",
        "conv15.bias": "rev_block5_conv1_bias_0.npy",
        "conv16.weight": "rev_block4_conv3_kernel_0.npy",
        "conv16.bias": "rev_block4_conv3_bias_0.npy",
        "conv17.weight": "rev_block4_conv2_kernel_0.npy",
        "conv17.bias": "rev_block4_conv2_bias_0.npy",
        "conv18.weight": "rev_block4_conv1_kernel_0.npy",
        "conv18.bias": "rev_block4_conv1_bias_0.npy",
        "conv19.weight": "rev_block3_conv3_kernel_0.npy",
        "conv19.bias": "rev_block3_conv3_bias_0.npy",
        "conv20.weight": "rev_block3_conv2_kernel_0.npy",
        "conv20.bias": "rev_block3_conv2_bias_0.npy",
        "conv21.weight": "rev_block3_conv1_kernel_0.npy",
        "conv21.bias": "rev_block3_conv1_bias_0.npy",
        "conv23.weight": "rev_block2_conv2_kernel_0.npy",
        "conv23.bias": "rev_block2_conv2_bias_0.npy",
        "conv24.weight": "rev_block2_conv1_kernel_0.npy",
        "conv24.bias": "rev_block2_conv1_bias_0.npy",
        "conv25.weight": "rev_block1_conv2_kernel_0.npy",
        "conv25.bias": "rev_block1_conv2_bias_0.npy",
        "conv26.weight": "rev_block1_conv1_kernel_0.npy",
        "conv26.bias": "rev_block1_conv1_bias_0.npy"
    }

    # Load weights into the PyTorch model
    for param_name, npy_file in weight_files.items():
        # Get parameter tensor from model
        param = getattr(torch_model, param_name.split(".")[0])
        param_weight = torch.from_numpy(np.load(os.path.join(weights_dir, npy_file)))

        # Transpose weights if it's a convolutional weight
        if 'weight' in param_name and len(param_weight.shape) == 4:
            param_weight = param_weight.permute(3, 2, 0, 1)

        # Assign weights to model
        if 'weight' in param_name:
            if param.weight.data.shape == param_weight.shape:
                param.weight.data.copy_(param_weight)
            else:
                print(f"Skipping {param_name} due to shape mismatch: {param.weight.data.shape} vs {param_weight.shape}")
        elif 'bias' in param_name:
            if param.bias.data.shape == param_weight.shape:
                param.bias.data.copy_(param_weight)
            else:
                print(f"Skipping {param_name} due to shape mismatch: {param.bias.data.shape} vs {param_weight.shape}")

        print(f"Loaded weights for {param_name}")

load_weights(torch_model, weights_dir)

# Step 4: Save the Model as .pth File
print("\nSaving model state dictionary...")
output_path = "models/dec_r51.pth"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
torch.save(torch_model.state_dict(), output_path)

print(f"Model weights saved to {output_path}")

# Optional: Verify the output
print("\nVerifying the output...")
input_tensor = torch.randn(1, 512, 7, 7)  # Input tensor for testing (match the expected input shape)
output = torch_model(input_tensor)
print("Output shape:", output.shape)


Loading TensorFlow model...
Extracting and saving weights...
Saved rev_block5_conv2_kernel_0 with shape (3, 3, 512, 512)
Saved rev_block5_conv2_bias_0 with shape (512,)
Saved rev_block5_conv1_kernel_0 with shape (3, 3, 512, 512)
Saved rev_block5_conv1_bias_0 with shape (512,)
Saved rev_block4_conv4_kernel_0 with shape (3, 3, 512, 512)
Saved rev_block4_conv4_bias_0 with shape (512,)
Saved rev_block4_conv3_kernel_0 with shape (3, 3, 512, 512)
Saved rev_block4_conv3_bias_0 with shape (512,)
Saved rev_block4_conv2_kernel_0 with shape (3, 3, 512, 512)
Saved rev_block4_conv2_bias_0 with shape (512,)
Saved rev_block4_conv1_kernel_0 with shape (3, 3, 512, 256)
Saved rev_block4_conv1_bias_0 with shape (256,)
Saved rev_block3_conv4_kernel_0 with shape (3, 3, 256, 256)
Saved rev_block3_conv4_bias_0 with shape (256,)
Saved rev_block3_conv3_kernel_0 with shape (3, 3, 256, 256)
Saved rev_block3_conv3_bias_0 with shape (256,)
Saved rev_block3_conv2_kernel_0 with shape (3, 3, 256, 256)
Saved rev_block

In [20]:
import torch
import torch.nn as nn

# Define the encoder or decoder models based on your model definition
class EncoderR11(nn.Module):
    def __init__(self):
        super(EncoderR11, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        return x

# Define the model for which you want to verify the state dictionary
def create_model(layer_name):
    if layer_name == "encoder_r11":
        return EncoderR11()
    # Add other encoder and decoder class definitions as needed
    # (Example, EncoderR21, DecoderR11, etc.)
    # Example: return EncoderR21() for `encoder_r21`

# Load the saved state dictionary from file
saved_state_dict_path = 'models/vgg_r11.pth'  # Replace with the actual path
saved_state_dict = torch.load(saved_state_dict_path)

# Initialize the model
layer_name = 'encoder_r11'  # Replace with the specific layer you want to check
model = create_model(layer_name)

# Get the state dictionary of the model
model_state_dict = model.state_dict()

# Rename keys in saved state dictionary to match the model state dictionary keys
renamed_state_dict = {}
for key in saved_state_dict.keys():
    if key.startswith("conv1_1"):
        new_key = key.replace("conv1_1", "conv1")  # Rename 'conv1_1' to 'conv1' to match model key
    elif key.startswith("conv1_2"):
        new_key = key.replace("conv1_2", "conv2")  # Adjust if other layers are present
    else:
        new_key = key  # If no change is required, use the original key

    renamed_state_dict[new_key] = saved_state_dict[key]

# Verify if all keys match after renaming
if renamed_state_dict.keys() != model_state_dict.keys():
    print("The keys in the renamed state dictionary do not match the model's state dictionary.")
    print("Renamed state keys:", renamed_state_dict.keys())
    print("Model state keys:", model_state_dict.keys())
else:
    print("The keys in the renamed state dictionary match the model's state dictionary.")

# Check if the shapes of each parameter match after renaming
for key in model_state_dict.keys():
    if key in renamed_state_dict:
        saved_shape = renamed_state_dict[key].shape
        model_shape = model_state_dict[key].shape
        if saved_shape != model_shape:
            print(f"Mismatch found in key: {key}")
            print(f"Saved state shape: {saved_shape}, Model state shape: {model_shape}")
        else:
            print(f"Parameter {key} matches in shape: {saved_shape}")
    else:
        print(f"Key {key} not found in renamed state dictionary.")

# Load the renamed state dict into the model
try:
    model.load_state_dict(renamed_state_dict)
    print("Model weights successfully loaded.")
except Exception as e:
    print(f"Error loading model weights: {e}")

print("Verification Complete.")


The keys in the renamed state dictionary do not match the model's state dictionary.
Renamed state keys: dict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias'])
Model state keys: odict_keys(['conv1.weight', 'conv1.bias'])
Parameter conv1.weight matches in shape: torch.Size([64, 3, 3, 3])
Parameter conv1.bias matches in shape: torch.Size([64])
Error loading model weights: Error(s) in loading state_dict for EncoderR11:
	Unexpected key(s) in state_dict: "conv2.weight", "conv2.bias". 
Verification Complete.


/tmp/ipykernel_278137/79259891.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(saved_state_dict_path)


In [16]:
import torch
import torch.nn as nn

# Load the saved state dictionary from the .pth file
saved_state_dict_path = 'models/vgg_r11.pth'  # Replace with the correct path
saved_state_dict = torch.load(saved_state_dict_path)

# Define a function to rename keys in the saved state dictionary to match the model's keys
def rename_keys(saved_state_dict):
    new_state_dict = {}
    for key in saved_state_dict.keys():
        if 'conv1_1' in key:
            new_key = key.replace('conv1_1', 'conv1')  # Rename to match model definition
        elif 'conv1_2' in key:
            new_key = key.replace('conv1_2', 'conv2')  # Update the conv layer name accordingly
        else:
            new_key = key  # If the key does not need renaming, leave it as it is
        new_state_dict[new_key] = saved_state_dict[key]
    return new_state_dict

# Rename keys in the saved state dictionary
renamed_state_dict = rename_keys(saved_state_dict)

# Define the model to load the weights into
class EncoderR11(nn.Module):
    def __init__(self):
        super(EncoderR11, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        return x

# Initialize the model
model = EncoderR11()

# Load the renamed state dictionary into the model
model.load_state_dict(renamed_state_dict)

# Confirm that the weights have been loaded correctly
print("Model weights loaded successfully.")

# Optional: Save the updated model with renamed keys for future use
torch.save(model.state_dict(), 'updated_model_r11.pth')
print("Updated model state dictionary saved to 'updated_model_r11.pth'")


Model weights loaded successfully.
Updated model state dictionary saved to 'updated_model_r11.pth'


/tmp/ipykernel_278137/4068817227.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(saved_state_dict_path)


In [18]:
import torch
import torch.nn as nn

# Load the saved state dictionary from the .pth file
saved_state_dict_path = 'models/vgg_r21.pth'  # Replace with the correct path
saved_state_dict = torch.load(saved_state_dict_path)

# Define a function to rename keys in the saved state dictionary to match the model's keys
def rename_keys(saved_state_dict):
    new_state_dict = {}
    for key in saved_state_dict.keys():
        if 'conv2_1' in key:
            new_key = key.replace('conv2_1', 'conv1')  # Rename to match model definition for layer 1
        elif 'conv2_2' in key:
            new_key = key.replace('conv2_2', 'conv2')  # Rename to match model definition for layer 2
        else:
            new_key = key  # If the key does not need renaming, leave it as it is
        new_state_dict[new_key] = saved_state_dict[key]
    return new_state_dict

# Rename keys in the saved state dictionary
renamed_state_dict = rename_keys(saved_state_dict)

# Define the model to load the weights into
class EncoderR21(nn.Module):
    def __init__(self):
        super(EncoderR21, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        self.relu3 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        return x

# Initialize the model
model = EncoderR21()

# Load the renamed state dictionary into the model
try:
    model.load_state_dict(renamed_state_dict)
    print("Model weights loaded successfully.")
except RuntimeError as e:
    print(f"Error loading weights: {e}")

# Optional: Save the updated model with renamed keys for future use
torch.save(model.state_dict(), 'updated_model_r21.pth')
print("Updated model state dictionary saved to 'updated_model_r21.pth'")


Error loading weights: Error(s) in loading state_dict for EncoderR21:
	Missing key(s) in state_dict: "conv2.weight", "conv2.bias", "conv3.weight", "conv3.bias". 
	Unexpected key(s) in state_dict: "conv1_1.weight", "conv1_1.bias", "conv1_2.weight", "conv1_2.bias". 
	size mismatch for conv1.weight: copying a param with shape torch.Size([128, 64, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 3, 3, 3]).
	size mismatch for conv1.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64]).
Updated model state dictionary saved to 'updated_model_r21.pth'


/tmp/ipykernel_278137/623198150.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(saved_state_dict_path)


In [19]:
import torch
import torch.nn as nn

# Load the saved state dictionary from the .pth file
saved_state_dict_path = 'models/vgg_r21.pth'  # Replace with the correct path
saved_state_dict = torch.load(saved_state_dict_path)

# Define a function to rename keys in the saved state dictionary to match the model's keys
def rename_keys(saved_state_dict):
    new_state_dict = {}
    for key in saved_state_dict.keys():
        if 'conv1_1' in key:
            new_key = key.replace('conv1_1', 'conv1')  # Rename to match model definition for layer 1
        elif 'conv1_2' in key:
            new_key = key.replace('conv1_2', 'conv2')  # Rename to match model definition for layer 2
        elif 'conv2_1' in key:
            new_key = key.replace('conv2_1', 'conv3')  # Rename to match model definition for layer 3
        else:
            new_key = key  # If the key does not need renaming, leave it as it is
        new_state_dict[new_key] = saved_state_dict[key]
    return new_state_dict

# Rename keys in the saved state dictionary
renamed_state_dict = rename_keys(saved_state_dict)

# Define the EncoderR21 Model
class EncoderR21(nn.Module):
    def __init__(self):
        super(EncoderR21, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(128, 256, 3, stride=1, padding=1)  # Adjusted for matching the size of loaded weights
        self.relu3 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        return x

# Initialize the model
model = EncoderR21()

# Handle Size Mismatch Manually
for name, param in model.named_parameters():
    if name in renamed_state_dict:
        saved_param = renamed_state_dict[name]
        if param.size() == saved_param.size():
            param.data.copy_(saved_param)
            print(f"Loaded weights for {name}")
        else:
            print(f"Size mismatch for {name}: expected {param.size()}, got {saved_param.size()}")

# Optional: Save the updated model with renamed keys for future use
torch.save(model.state_dict(), 'updated_model_r21.pth')
print("Updated model state dictionary saved to 'updated_model_r21.pth'")


Loaded weights for conv1.weight
Loaded weights for conv1.bias
Size mismatch for conv2.weight: expected torch.Size([128, 64, 3, 3]), got torch.Size([64, 64, 3, 3])
Size mismatch for conv2.bias: expected torch.Size([128]), got torch.Size([64])
Size mismatch for conv3.weight: expected torch.Size([256, 128, 3, 3]), got torch.Size([128, 64, 3, 3])
Size mismatch for conv3.bias: expected torch.Size([256]), got torch.Size([128])
Updated model state dictionary saved to 'updated_model_r21.pth'


/tmp/ipykernel_278137/4235575760.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(saved_state_dict_path)


In [1]:
import torch
import torch.nn as nn

# Define the encoder or decoder models based on your model definition
class EncoderR11(nn.Module):
    def __init__(self):
        super(EncoderR11, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        return x

# Define the model for which you want to verify and update the state dictionary
def create_model(layer_name):
    if layer_name == "encoder_r11":
        return EncoderR11()
    # Add other encoder and decoder class definitions as needed
    return None

# Load the saved state dictionary from file
saved_state_dict_path = 'models/vgg_r11.pth'  # Replace with the actual path
saved_state_dict = torch.load(saved_state_dict_path)

# Initialize the model
layer_name = 'encoder_r11'  # Replace with the specific layer you want to check
model = create_model(layer_name)

# Rename keys in the saved state dictionary to match the model's state dictionary keys
renamed_state_dict = {}
for key in saved_state_dict.keys():
    if key.startswith("conv1_1"):
        new_key = key.replace("conv1_1", "conv1")  # Rename 'conv1_1' to 'conv1' to match model key
    elif key.startswith("conv1_2"):
        # Since `conv1_2` isn't present in `EncoderR11`, we will skip adding it.
        continue
    else:
        new_key = key  # If no change is required, use the original key

    renamed_state_dict[new_key] = saved_state_dict[key]

# Filter out keys that do not exist in the model's state dictionary
filtered_state_dict = {k: v for k, v in renamed_state_dict.items() if k in model.state_dict()}

# Check if all keys match after renaming and filtering
if filtered_state_dict.keys() != model.state_dict().keys():
    print("The keys in the filtered state dictionary do not fully match the model's state dictionary.")
    print("Filtered state keys:", filtered_state_dict.keys())
    print("Model state keys:", model.state_dict().keys())
else:
    print("The keys in the filtered state dictionary match the model's state dictionary.")

# Check if the shapes of each parameter match after filtering
for key in filtered_state_dict.keys():
    if key in model.state_dict():
        saved_shape = filtered_state_dict[key].shape
        model_shape = model.state_dict()[key].shape
        if saved_shape != model_shape:
            print(f"Mismatch found in key: {key}")
            print(f"Saved state shape: {saved_shape}, Model state shape: {model_shape}")
        else:
            print(f"Parameter {key} matches in shape: {saved_shape}")

# Load the filtered state dict into the model
try:
    model.load_state_dict(filtered_state_dict)
    print("Model weights successfully loaded.")
except Exception as e:
    print(f"Error loading model weights: {e}")

print("Verification Complete.")


The keys in the filtered state dictionary match the model's state dictionary.
Parameter conv1.weight matches in shape: torch.Size([64, 3, 3, 3])
Parameter conv1.bias matches in shape: torch.Size([64])
Model weights successfully loaded.
Verification Complete.


/tmp/ipykernel_298324/2868298935.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(saved_state_dict_path)


In [2]:
saved_state_dict = torch.load(saved_state_dict_path, weights_only=True)
torch.save(model.state_dict(), 'updated_model_r11.pth')
